In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    Context,
    step
)

from llama_index.llms.openai import OpenAI

In [3]:
import json

class VerifyCodeEvent(Event):
    code_sample : str

class CodeAnalysisEvent(Event):
    code_analysis : dict

class SearchInWebEvent(Event):
    pass

class ValidateResultEvent(Event):
    pass

class NotFoundEvent(Event):
    pass

class UpdateDBEvent(Event):
    pass

class SearchEventResult(Event):
    pass


In [4]:
from llama_index.core.agent import ReActAgent
import json, os

class InfoFusionWorkflow(Workflow):

    @step(pass_context = True)
    async def take_user_input(self, ctx : Context, ev : StartEvent | CodeAnalysisEvent) -> VerifyCodeEvent | StopEvent:
        if isinstance(ev, StartEvent):
            if hasattr(ev, 'query'):
                await ctx.set('query', ev.query)

            if hasattr(ev, 'llm'):
                await ctx.set('llm', ev.llm)

            if hasattr(ev, 'tools'):
                await ctx.set('tools', ev.tools)

            human_prompt = """
                Enter your code for verification or press q/Q to quit.
    """
        else:
            if 'response' in ev.code_analysis:
                human_prompt = str(ev.code_analysis)
            else: 
                human_prompt = """
                Here is the suggestion for your code.
                
                {str(ev.code_analysis)}

                Modify the code as per the suggessions and submit the code. If you want to quit press q/Q.
"""

        human_input = input (human_prompt)
        print(f"HI:{human_input}")

        quit_str = "q"
        human_input = human_input.strip()
        print(f"HI:*{human_input}*, *{quit_str}*")
        if human_input.lower() is quit_str:
            print("sending stop")
            ctx.send_event(StopEvent("User decided to stop"))

        ctx.send_event(VerifyCodeEvent(code_sample = human_input))

   
    @step(pass_context = True)
    async def verify_code(self, ctx : Context, ev : VerifyCodeEvent) -> CodeAnalysisEvent: #QueryEvent:
        
        prompt = f"""
Given a code sample verify that the code is correct. Do not provide an answer directly, instead identify the mistakes. 
Provide some reference articals so that the user can fix the error by themselve. Perform a web-search, donot makeup any information.

Respond in pure JSON without any markdown, like this :
{{
    "Code_Analysis" : {{
        "code_sample" : "",
        "code_language" : "",
        "code_mistakes" : [
        {{"mistake1", "reference1"}},
        {{"mistake2", "reference2"}},
        ]
    }}
}}

if there are no mistakes in the code, respond in pure JSON without any markdown, like this:
{{
    "Code_Analysis" : {{
        "code_sample" : "",
        "code_language" : "",
        "response" : "Congratulations on successfully writing a correct code."
        }}
}}

Here is the user code : {ev.code_sample}
"""
        agent = ReActAgent.from_tools(tools = [], llm = await ctx.get('llm'), verbose = True)
        response = agent.chat(prompt)
        # llm = await ctx.get('llm')
        # response = llm.complete(prompt)
        print(f"\nResponse: {response}")

        response_json = json.loads(str(response))

        print(f"json:\n{response_json}\n")

        
        ctx.send_event(CodeAnalysisEvent(code_analysis = response_json))

    # @step(pass_context = True)
    # async def query_router(self, ctx : Context, ev : QueryEvent) -> StopEvent: #SearchInWebEvent: #| SearchInDBEvent :
    #     pass
        

    # @step(pass_context = True)
    # async def search_in_db(self, ev : SearchInDBEvent) -> ValidateResultEvent | NotFoundEvent:
    #     pass

    # @step(pass_context = True)
    # async def search_in_web(self, ev : SearchInWebEvent) -> ValidateResultEvent:
    #     pass

    # @step(pass_context = True)
    # async def update_db(self, ev : UpdateDBEvent) -> StopEvent:
    #     pass

    # @step(pass_context = True)
    # async def generate_final_result(self, ev : SearchEventResult) -> StopEvent | None:
    #     pass

In [5]:
import os, traceback

from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key=os.environ['TAVILY_API_KEY'],
)


llm = OpenAI(model = "gpt-4o-mini", temperature = 0)
workflow = InfoFusionWorkflow(timeout = 60, verbose = True)

tools = tavily_tool.to_tool_list()
code_sample = """
int main() {
    i = 20
    print(i)

    return 0
}
"""
try:
    result = await workflow.run(llm = llm,
                                tools = tools,
                                code_sample = code_sample)

    print(result)
except Exception as e:
    print(f"Exception: {e} \n {traceback.format_exc()}")


Running step take_user_input
HI:q
HI:*q*, *q*
Step take_user_input produced no event
Running step verify_code
> Running step 1fca57cf-0625-4f24-8d5c-403558f2d71f. Step input: 
Given a code sample verify that the code is correct. Do not provide an answer directly, instead identify the mistakes. 
Provide some reference articals so that the user can fix the error by themselve. Perform a web-search, donot makeup any information.

Respond in pure JSON without any markdown, like this :
{
    "Code_Analysis" : {
        "code_sample" : "",
        "code_language" : "",
        "code_mistakes" : [
        {"mistake1", "reference1"},
        {"mistake2", "reference2"},
        ]
    }
}

if there are no mistakes in the code, respond in pure JSON without any markdown, like this:
{
    "Code_Analysis" : {
        "code_sample" : "",
        "code_language" : "",
        "response" : "Congratulations on successfully writing a correct code."
        }
}

Here is the user code : q

Thought: (Implici

In [ ]:
class WorkflowAdapter:
    def __init__(self):
        import os, traceback

        from llama_index.tools.tavily_research import TavilyToolSpec

        tavily_tool = TavilyToolSpec(
            api_key=os.environ['TAVILY_API_KEY'],
        )


        self.llm = OpenAI(model = "gpt-4o-mini", temperature = 0)
        self.tools = tavily_tool.to_tool_list()

    async def verify_code(self, code_sample):
        workflow = InfoFusionWorkflow(timeout = 60, verbose = True)

        result = None
        try:
            result = await workflow.run(llm = self.llm,
                                        tools = self.tools,
                                        code_sample = code_sample)

            print(result)
        except Exception as e:
            print(f"Exception: {e} \n {traceback.format_exc()}")
            result = "Exception : {e}"
            
        return result


In [11]:
# 